In [1]:
# would like to have a way of creating __slots__, properties, additional methods for each class
# properties and slots are will be created for fields in _fields

class Point2D:
    _fields = ["x", "y"]

    def __init__(self, x, y):
        self._x = x
        self._y = y


class Point3D:
    _fields = ["x", "y", "z"]

    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z


In [2]:
class SlottedStruct(type):
    def __new__(mcls, name, bases, cls_dict):
        cls_object = super().__new__(mcls, name, bases, cls_dict)
        # __slots__
        setattr(cls_object, "__slots__", [f"_{field}" for field in cls_object._fields])

        # read-only properties
        for field in cls_object._fields:
            slot = f"_{field}"
            # one way of setting the properties
            # lambda is a closure - it will share `slot` with every lambda instance
            # and will have the same `slot` value (last slot from the loop) for each lambda 
            # setattr(cls_object, field, property(fget=lambda self: getattr(self, slot)))

            # instead value of `slot` should be calculated when lambda is being created
            # if we set a default for each lambda, it will not change later
            setattr(cls_object, field, property(fget=lambda self, slot_name=slot: getattr(self, slot_name)))

        return cls_object

In [3]:
class Person(metaclass=SlottedStruct):
    _fields = ["name", "age"]
    def __init__(self, name, age):
        self._name = name
        self._age = age


In [4]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '_fields': ['name', 'age'],
              '__init__': <function __main__.Person.__init__(self, name, age)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              '__slots__': ['_name', '_age'],
              'name': <property at 0x1121871a0>,
              'age': <property at 0x1121871f0>})

In [5]:
p = Person("Bob", 33)
p.__dict__

{'_name': 'Bob', '_age': 33}

In [6]:
p.name, p.age

('Bob', 33)

In [7]:
try:
    p.name = "John"
except AttributeError as e:
    print(e)

property of 'Person' object has no setter


In [8]:
class SlottedStruct(type):
    def __new__(mcls, name, bases, cls_dict):
        cls_object = super().__new__(mcls, name, bases, cls_dict)

        # __slots__
        setattr(cls_object, "__slots__", [f"_{field}" for field in cls_object._fields])

        # read-only properties
        for field in cls_object._fields:
            slot = f"_{field}"
            setattr(cls_object, field, property(fget=lambda self, slot_name=slot: getattr(self, slot_name)))

        # these functions below are closures!
        
        # __eq__
        def eq(self, other):
            if isinstance(other, cls_object):
                self_fields = [getattr(self, field) for field in cls_object._fields]
                other_fields = [getattr(other, field) for field in cls_object._fields]
                return self_fields == other_fields
            return False
        setattr(cls_object, "__eq__", eq)

        # __hash__
        def hash_(self):
            # using tuple, as lists are not hashable
            self_fields = tuple(getattr(self, field) for field in cls_object._fields)
            return hash(self_fields)
        setattr(cls_object, "__hash__", hash_)

        # __str__
        def str_(self):
            self_fields = [getattr(self, field) for field in cls_object._fields]
            fields_values = ", ".join(str(field) for field in self_fields)
            return f"{cls_object.__name__}({fields_values})"
        setattr(cls_object, "__str__", str_)

        # __repr__ - uses `self` instead of `cls_object` - works as well
        def repr_(self):
            fields_values = ", ".join(f"{field}={getattr(self, field)}" for field in self._fields)
            return f"{self.__class__.__name__}({fields_values})"
        setattr(cls_object, "__repr__", repr_)

        return cls_object



In [9]:
class Person(metaclass=SlottedStruct):
    _fields = ["name"]

    def __init__(self, name):
        self._name = name


In [10]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '_fields': ['name'],
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              '__slots__': ['_name'],
              'name': <property at 0x112187330>,
              '__eq__': <function __main__.SlottedStruct.__new__.<locals>.eq(self, other)>,
              '__hash__': <function __main__.SlottedStruct.__new__.<locals>.hash_(self)>,
              '__str__': <function __main__.SlottedStruct.__new__.<locals>.str_(self)>,
              '__repr__': <function __main__.SlottedStruct.__new__.<locals>.repr_(self)>})

In [11]:
p1 = Person("Bob")
p2 = Person("Bob")
p1 is p2, p1 == p2, isinstance(p1, Person), isinstance(p2, Person)

(False, True, True, True)

In [12]:
hash(p1) == hash(p2)

True

In [13]:
str(p1), repr(p1)

('Person(Bob)', 'Person(name=Bob)')

In [14]:
p3 = Person("Alex")
p2.name, p3.name

('Bob', 'Alex')

In [15]:
# now we don't need to define properties etc
# it's also possible to inherit it

class Point2D(metaclass=SlottedStruct):
    _fields = ["x", "y"]

    def __init__(self, x, y):
        self._x = x
        self._y = y


class Point3D(metaclass=SlottedStruct):
    _fields = ["x", "y", "z"]

    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

In [16]:
p1 = Point2D(1, 1)
p2 = Point2D(2, 3)
p3 = Point3D(3, 4, 5)

repr(p1), str(p1), repr(p3), str(p3), type(p3.__eq__)

('Point2D(x=1, y=1)',
 'Point2D(1, 1)',
 'Point3D(x=3, y=4, z=5)',
 'Point3D(3, 4, 5)',
 method)

In [17]:
p1 == p2, p2 == p3

(False, False)

In [18]:
Point2D.__name__, Point2D.__bases__, Point2D.__dict__

('Point2D',
 (object,),
 mappingproxy({'__module__': '__main__',
               '_fields': ['x', 'y'],
               '__init__': <function __main__.Point2D.__init__(self, x, y)>,
               '__dict__': <attribute '__dict__' of 'Point2D' objects>,
               '__weakref__': <attribute '__weakref__' of 'Point2D' objects>,
               '__doc__': None,
               '__slots__': ['_x', '_y'],
               'x': <property at 0x1121b4d60>,
               'y': <property at 0x1121b4d10>,
               '__eq__': <function __main__.SlottedStruct.__new__.<locals>.eq(self, other)>,
               '__hash__': <function __main__.SlottedStruct.__new__.<locals>.hash_(self)>,
               '__str__': <function __main__.SlottedStruct.__new__.<locals>.str_(self)>,
               '__repr__': <function __main__.SlottedStruct.__new__.<locals>.repr_(self)>}))

In [19]:
def struct(cls):
    """
    Class decorator to use the metaclass.
    Remember how decorators work:
    cls = struct(cls)  # now cls is an instance of our metaclass, which is a regular class!
    """
    return SlottedStruct(cls.__name__, cls.__bases__, dict(cls.__dict__))  # instance of a metaclass, which is a regular class!


@struct
class Point2D:
    _fields = ["x", "y"]  # dataclass could be used instead

    def __init__(self, x, y):
        self._x = x
        self._y = y


@struct
class Point3D:
    _fields = ["x", "y", "z"]

    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z



In [20]:
p1 = Point2D(1, 1)
p2 = Point2D(2, 3)
p3 = Point3D(3, 4, 5)

repr(p1), str(p1), repr(p3), str(p3)

('Point2D(x=1, y=1)',
 'Point2D(1, 1)',
 'Point3D(x=3, y=4, z=5)',
 'Point3D(3, 4, 5)')

In [21]:
p1.__slots__

['_x', '_y']

In [22]:
try:
    p1.__dict__  # __slots__ are used just as defined inside the metaclass
except TypeError as e:
    print(e)

descriptor '__dict__' for 'Point2D' objects doesn't apply to a 'Point2D' object
